###  first selection

- text-generation []
- decoder []

- open-source models [x]
- HF availability [x]
- Task: text generation [x]
- Date []
- architecture: decoder-based only,  []
- model size < SLMs size [x]
  - https://huggingface.co/docs/accelerate/main/en/usage_guides/model_size_estimator?
  - model size from endpoint?
  - safetensors -> model_info.safetensors -> https://huggingface.co/microsoft/phi-2 [x]
  - check it matches our data about models used []

- [?]Code generation [] used in code generation, trained in code datasets?


In [1]:
from huggingface_hub import HfApi
import pandas as pd


In [2]:
pwd

'/home/fjdur/energy-ml-serving'

In [3]:
import sys
sys.path.append("model_selection/") 

In [4]:
save_dir = 'model_selection/tmp/'

## Get first dataset

In [17]:
# Initialize API
api = HfApi()

# Fetch models metadata
limit = 20
models = api.list_models(task="text-generation", pipeline_tag="text-generation",sort="downloads", limit=limit)
#models = api.list_models(task="text-generation", sort="downloads", limit=500)



In [4]:
import pandas as pd

# Filter for decoder-only architectures
decoder_models = []
i = 0
num_models = 200000

for model in models:
    print(f"{i}-->")
    if i > num_models:
        print("breaking")
        break

    model_id = getattr(model, "modelId", "Unknown")
    print(f'model_id: {model_id}')

    # Fetch detailed model information
    model_info = api.model_info(model.modelId)
    print(f'config: {model_info.config}')
    print(f'd1: {model_info.downloads_all_time}')
    print(f'd2: {model_info.downloads}')
    #print(f'gguf: {model_info.gguf}')
    print(f'safetensors: {model_info.safetensors}')



    if model_info.config ==None:
        continue

    # Collect model metadata with the desired column names
    model_metadata = {
        #"model": model.modelId,  # Model name or ID
        "model": getattr(model, "modelId", "Unknown"),

        "id": getattr(model, "modelId", "Unknown"),
        #"downloads_all_time": getattr(model_info, "downloads_all_time", "Unknown"),
        "downloads": getattr(model_info, "downloads", "Unknown"),
        "safetensors": getattr(model_info, "safetensors", "Unknown"),

        "tags": getattr(model, "tags", "Unknown"),
        "task": getattr(model_info, "pipeline_tag", "Unknown"),
        "card_data": getattr(model_info, "card_data", "Unknown"),
        "created_at": getattr(model, "created_at", "Unknown"),
        "library_name": getattr(model, "library_name", "Unknown"),
        "transformers_info": getattr(model_info, "transformers_info", "Unknown"),
        "architecture": model_info.config.get("architectures", "Unknown")
    }
    print(f'model_metadata: {model_metadata}')

    decoder_models.append(model_metadata)
    i += 1




0-->
model_id: openai-community/gpt2
config: {'architectures': ['GPT2LMHeadModel'], 'model_type': 'gpt2', 'tokenizer_config': {}}


AttributeError: 'ModelInfo' object has no attribute 'downloads_all_time'

In [72]:
# Check for inconsistencies
for index, item in enumerate(decoder_models):
    if not isinstance(item, dict):
        print(f"Item at index {index} is not a dictionary: {item}")
    else:
        # Check for problematic keys or data
        for key, value in item.items():
            if value is None:
                print(f"Key '{key}' in item at index {index} has a value of None.")


Key 'safetensors' in item at index 5 has a value of None.
Key 'safetensors' in item at index 6 has a value of None.


In [5]:
# Attempt to create the DataFrame in a simplified way
try:
    df = pd.DataFrame(decoder_models)
    print("DataFrame created successfully")
    #print(df.head())  # Display only the first few rows to avoid rendering issues
except Exception as e:
    print(f"Error creating DataFrame: {e}")

DataFrame created successfully


In [7]:
df

""


In [ ]:
# used python script

#df.to_csv("decoder_models.csv", index=False)
#print("CSV file saved: decoder_models.csv")

CSV file saved: decoder_models.csv


In [46]:
df.columns

Index(['model', 'id', 'downloads', 'safetensors', 'tags', 'task', 'created_at',
       'library_name', 'transformers_info', 'architecture'],
      dtype='object')

In [47]:
df['created_at']

0    2024-09-17 14:10:29+00:00
1    2022-03-02 23:29:04+00:00
2    2024-07-16 01:29:54+00:00
3    2024-07-16 08:07:29+00:00
4    2022-10-08 16:14:42+00:00
5    2022-05-11 08:25:17+00:00
6    2024-07-18 08:56:00+00:00
7    2022-03-02 23:29:04+00:00
8    2024-07-16 16:07:46+00:00
9    2024-04-17 09:35:12+00:00
10   2023-02-08 15:12:33+00:00
11   2023-03-29 07:11:13+00:00
12   2024-09-18 15:12:47+00:00
13   2024-07-04 10:15:41+00:00
14   2023-07-13 16:16:13+00:00
15   2024-09-18 15:03:14+00:00
16   2023-12-30 06:27:30+00:00
17   2024-10-22 15:44:28+00:00
18   2023-06-17 19:52:11+00:00
19   2022-03-02 23:29:04+00:00
Name: created_at, dtype: datetime64[ns, UTC]

## Read file

In [5]:
import math
import re

In [6]:
file_name = 'model_selection/results/decoder_models_info_02.csv'
df = pd.read_csv(file_name)
print(f'total models: {len(df)}')

total models: 104816


In [7]:
df.iloc[0]['safetensors']
df.iloc[0]['eval_results']


nan

In [8]:
df.columns

Index(['model', 'id', 'created_at', 'downloads_all_time', 'downloads', 'likes',
       'safetensors', 'task', 'datasets', 'eval_results', 'metrics',
       'library_name', 'transformers_info', 'architecture', 'tags',
       'card_data'],
      dtype='object')

In [9]:

# Filter out rows where the 'safetensors' column is NaN
filtered_df = df[df['safetensors'].notna()]

# Display the filtered DataFrame
print("After safetensors check: ",len(filtered_df))

# Filter out rows where the 'safetensors' column is NaN
filtered_df = filtered_df[filtered_df['eval_results'].notna() ]

print("After eval_results check: ",len(filtered_df))

# # Ensure 'eval_results' is not an empty list
# filtered_df = filtered_df[
#     filtered_df['eval_results'].apply(lambda x: not isinstance(x, list) or (isinstance(x, list) and len(x) > 0) or (isinstance(x, str) and x != "[]"))
# ]


# Display the filtered DataFrame
print(len(filtered_df))

# Ensure 'eval_results' is not an empty list or stringified empty list
filtered_df = filtered_df[
    filtered_df['eval_results'].apply(
        lambda x: not (isinstance(x, list) and len(x) == 0) and x != "[]" and pd.notna(x)
    )
]

print("After eval_results second check: ",len(filtered_df))

# Display the length of the filtered DataFrame
print(len(filtered_df))


save_evaluated_df =False
if save_evaluated_df:
    # Optional: Save the filtered DataFrame to a CSV file
    filtered_df.to_csv(save_dir + "evaluated_models.csv", index=False)
    print(f"Filtered CSV file saved: {save_dir}evaluated_models.csv")

After safetensors check:  67761
After eval_results check:  8011
8011
After eval_results second check:  1933
1933


In [10]:
print(type(filtered_df.iloc[2]['eval_results']))
print(filtered_df.iloc[2]['eval_results'])
print(len(filtered_df.iloc[2]['eval_results']))


<class 'str'>
[EvalResult(task_type='text-generation', dataset_type='openai_humaneval', dataset_name='humaneval', metric_type='pass@1', metric_value=0.15542682926829265, task_name=None, dataset_config=None, dataset_split=None, dataset_revision=None, dataset_args=None, metric_name='pass@1', metric_config=None, metric_args=None, verified=False, verify_token=None, source_name=None, source_url=None), EvalResult(task_type='text-generation', dataset_type='openai_humaneval', dataset_name='humaneval', metric_type='pass@10', metric_value=0.3278356276947017, task_name=None, dataset_config=None, dataset_split=None, dataset_revision=None, dataset_args=None, metric_name='pass@10', metric_config=None, metric_args=None, verified=False, verify_token=None, source_name=None, source_url=None), EvalResult(task_type='text-generation', dataset_type='openai_humaneval', dataset_name='humaneval', metric_type='pass@100', metric_value=0.5719815685597749, task_name=None, dataset_config=None, dataset_split=None, d

In [11]:
# Adding num_parameters column

# Function to extract the 'total' value from the 'safetensors' string
def extract_total(safetensors_info):
    if pd.isna(safetensors_info):
        return None
    match = re.search(r"total=(\d+)", safetensors_info)
    return int(match.group(1)) if match else None

# Create a new column 'num_parameters' with the extracted total
filtered_df['num_parameters'] = filtered_df['safetensors'].apply(extract_total)
filtered_df.head()

,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,metrics,library_name,transformers_info,architecture,tags,card_data,num_parameters
18,ibm-granite/granite-3.0-8b-instruct,ibm-granite/granite-3.0-8b-instruct,2024-10-02 21:16:23+00:00,Unknown,19349,148,SafeTensorsInfo(parameters={'BF16': 8170848256...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['GraniteForCausalLM'],"['transformers', 'safetensors', 'granite', 'te...",license: apache-2.0\nlibrary_name: transformer...,8170848256
40,arcee-ai/SuperNova-Medius,arcee-ai/SuperNova-Medius,2024-10-02 06:50:01+00:00,Unknown,7777,160,SafeTensorsInfo(parameters={'BF16': 1477003366...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['Qwen2ForCausalLM'],"['transformers', 'safetensors', 'qwen2', 'text...",license: apache-2.0\nlibrary_name: transformer...,14770033664
46,bigscience/bloom,bigscience/bloom,2022-05-19 11:53:33+00:00,Unknown,10666,4760,SafeTensorsInfo(parameters={'BF16': 1762472714...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['BloomForCausalLM'],"['transformers', 'pytorch', 'tensorboard', 'sa...",language:\n- ak\n- ar\n- as\n- bm\n- bn\n- ca\...,176247271424
64,dfurman/CalmeRys-78B-Orpo-v0.1,dfurman/CalmeRys-78B-Orpo-v0.1,2024-09-24 10:25:46+00:00,Unknown,5470,35,SafeTensorsInfo(parameters={'BF16': 7796546355...,text-generation,['mlabonne/orpo-dpo-mix-40k'],"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['Qwen2ForCausalLM'],"['transformers', 'safetensors', 'qwen2', 'text...",language:\n- en\nlicense: mit\nlibrary_name: t...,77965463552
72,Orenguteng/Llama-3.1-8B-Lexi-Uncensored-V2,Orenguteng/Llama-3.1-8B-Lexi-Uncensored-V2,2024-08-09 20:39:15+00:00,Unknown,29053,87,SafeTensorsInfo(parameters={'BF16': 8030261248...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",license: llama3.1\nlibrary_name: transformers\...,8030261248


In [12]:
# Adding popularity column: sum of normalized likes and downloads

# Normalize the 'likes' and 'downloads' columns
#filtered_df['normalized_likes'] = df['likes'] / df['likes'].max()
#df['normalized_downloads'] = df['downloads'] / df['downloads'].max()

# Calculate the sum of the normalized 'likes' and 'downloads'
print(f"likes max: {filtered_df['likes'].max()} downloads max: {filtered_df['downloads'].max()}")
filtered_df['popularity'] = (filtered_df['likes'] / filtered_df['likes'].max()) + (filtered_df['downloads'] / filtered_df['downloads'].max())
print(f"popularity max: {filtered_df['popularity'].max()} ")


filtered_df.head()

likes max: 4760 downloads max: 12174074
popularity max: 1.0220588235294117 


,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,metrics,library_name,transformers_info,architecture,tags,card_data,num_parameters,popularity
18,ibm-granite/granite-3.0-8b-instruct,ibm-granite/granite-3.0-8b-instruct,2024-10-02 21:16:23+00:00,Unknown,19349,148,SafeTensorsInfo(parameters={'BF16': 8170848256...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['GraniteForCausalLM'],"['transformers', 'safetensors', 'granite', 'te...",license: apache-2.0\nlibrary_name: transformer...,8170848256,0.032682
40,arcee-ai/SuperNova-Medius,arcee-ai/SuperNova-Medius,2024-10-02 06:50:01+00:00,Unknown,7777,160,SafeTensorsInfo(parameters={'BF16': 1477003366...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['Qwen2ForCausalLM'],"['transformers', 'safetensors', 'qwen2', 'text...",license: apache-2.0\nlibrary_name: transformer...,14770033664,0.034252
46,bigscience/bloom,bigscience/bloom,2022-05-19 11:53:33+00:00,Unknown,10666,4760,SafeTensorsInfo(parameters={'BF16': 1762472714...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['BloomForCausalLM'],"['transformers', 'pytorch', 'tensorboard', 'sa...",language:\n- ak\n- ar\n- as\n- bm\n- bn\n- ca\...,176247271424,1.000876
64,dfurman/CalmeRys-78B-Orpo-v0.1,dfurman/CalmeRys-78B-Orpo-v0.1,2024-09-24 10:25:46+00:00,Unknown,5470,35,SafeTensorsInfo(parameters={'BF16': 7796546355...,text-generation,['mlabonne/orpo-dpo-mix-40k'],"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['Qwen2ForCausalLM'],"['transformers', 'safetensors', 'qwen2', 'text...",language:\n- en\nlicense: mit\nlibrary_name: t...,77965463552,0.007802
72,Orenguteng/Llama-3.1-8B-Lexi-Uncensored-V2,Orenguteng/Llama-3.1-8B-Lexi-Uncensored-V2,2024-08-09 20:39:15+00:00,Unknown,29053,87,SafeTensorsInfo(parameters={'BF16': 8030261248...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",license: llama3.1\nlibrary_name: transformers\...,8030261248,0.020664


In [13]:
filtered_df['created_at'] = pd.to_datetime(filtered_df['created_at'], errors='coerce')

In [14]:
# If filter by any date

filter_date = False
if filter_date :
    # Filter models with 'created_at' in 2023 or later
    year = 2021
    filtered_df = filtered_df[filtered_df['created_at'].dt.year >= year]

    print(f"filtered w/ created_at len now:{len(filtered_df)}")
else:
    print("Considering all models without date filter")


Considering all models without date filter


In [15]:
# Filter out rows where the 'safetensors' column is NaN
limit_slm_parameters=5 *1e9 # 10B
print(len(filtered_df))
print(f'limit of num_parameters considered: {limit_slm_parameters}')
filtered_df = filtered_df[filtered_df['num_parameters'] <= limit_slm_parameters]

# Display the filtered DataFrame
print(len(filtered_df))
filtered_df.head()

1933
limit of num_parameters considered: 5000000000.0
597


,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,metrics,library_name,transformers_info,architecture,tags,card_data,num_parameters,popularity
89,ibm-granite/granite-3.0-2b-instruct,ibm-granite/granite-3.0-2b-instruct,2024-10-02 21:07:46+00:00,Unknown,11284,31,SafeTensorsInfo(parameters={'BF16': 2634201088...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['GraniteForCausalLM'],"['transformers', 'safetensors', 'granite', 'te...",license: apache-2.0\nlibrary_name: transformer...,2634201088,0.007439
90,Lyte/Llama-3.2-3B-Overthinker,Lyte/Llama-3.2-3B-Overthinker,2024-10-17 22:49:53+00:00,Unknown,717,18,SafeTensorsInfo(parameters={'F16': 3212749824}...,text-generation,['Lyte/Reasoning-Paused'],"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'll...",language:\n- en\nlicense: apache-2.0\ntags:\n-...,3212749824,0.003840
178,distilbert/distilgpt2,distilbert/distilgpt2,2022-03-02 23:29:04+00:00,Unknown,3973493,442,"SafeTensorsInfo(parameters={'F32': 88204032}, ...",text-generation,['openwebtext'],"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['GPT2LMHeadModel'],"['transformers', 'pytorch', 'tf', 'jax', 'tfli...",language: en\nlicense: apache-2.0\ntags:\n- ex...,88204032,0.419247
197,bigcode/starcoder2-3b,bigcode/starcoder2-3b,2023-11-29 15:22:51+00:00,Unknown,161709,146,SafeTensorsInfo(parameters={'F32': 3030371328}...,text-generation,['bigcode/the-stack-v2-train'],"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['Starcoder2ForCausalLM'],"['transformers', 'safetensors', 'starcoder2', ...",license: bigcode-openrail-m\nlibrary_name: tra...,3030371328,0.043955
313,facebook/layerskip-llama3.2-1B,facebook/layerskip-llama3.2-1B,2024-10-17 10:05:32+00:00,Unknown,471,15,SafeTensorsInfo(parameters={'BF16': 1235814400...,text-generation,NaN,"[EvalResult(task_type='question-answering', da...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'll...",language:\n- en\nlicense: other\nlibrary_name:...,1235814400,0.003190


In [16]:
import re

# Define a regex pattern to match 'dataset_name' containing 'humaneval' (case insensitive)
pattern = r"dataset_name=['\"]humaneval['\"]" # there are other like 'humanevalsynthesis'

# Filter rows where 'eval_results' matches the regex pattern
filtered_df = filtered_df[
    filtered_df['eval_results'].str.contains(pattern, case=False, na=False)
]

# Display the length of the filtered DataFrame
print("Filtered DataFrame Length:", len(filtered_df))

# Display the filtered DataFrame
filtered_df.head()

Filtered DataFrame Length: 69


,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,metrics,library_name,transformers_info,architecture,tags,card_data,num_parameters,popularity
197,bigcode/starcoder2-3b,bigcode/starcoder2-3b,2023-11-29 15:22:51+00:00,Unknown,161709,146,SafeTensorsInfo(parameters={'F32': 3030371328}...,text-generation,['bigcode/the-stack-v2-train'],"[EvalResult(task_type='text-generation', datas...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['Starcoder2ForCausalLM'],"['transformers', 'safetensors', 'starcoder2', ...",license: bigcode-openrail-m\nlibrary_name: tra...,3030371328,0.043955
313,facebook/layerskip-llama3.2-1B,facebook/layerskip-llama3.2-1B,2024-10-17 10:05:32+00:00,Unknown,471,15,SafeTensorsInfo(parameters={'BF16': 1235814400...,text-generation,NaN,"[EvalResult(task_type='question-answering', da...",NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'll...",language:\n- en\nlicense: other\nlibrary_name:...,1235814400,0.003190
443,bigscience/bloomz-560m,bigscience/bloomz-560m,2022-10-08 16:14:42+00:00,Unknown,12174074,105,"SafeTensorsInfo(parameters={'F16': 559214592},...",text-generation,['bigscience/xP3'],[EvalResult(task_type='Coreference resolution'...,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['BloomForCausalLM'],"['transformers', 'pytorch', 'tensorboard', 'sa...",language:\n- ak\n- ar\n- as\n- bm\n- bn\n- ca\...,559214592,1.022059
444,bigscience/bloomz-1b1,bigscience/bloomz-1b1,2022-10-08 16:16:01+00:00,Unknown,3627,32,SafeTensorsInfo(parameters={'F16': 1065314304}...,text-generation,['bigscience/xP3'],[EvalResult(task_type='Coreference resolution'...,NaN,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['BloomForCausalLM'],"['transformers', 'pytorch', 'tensorboard', 'sa...",language:\n- ak\n- ar\n- as\n- bm\n- bn\n- ca\...,1065314304,0.007021
565,bigcode/starcoderbase-1b,bigcode/starcoderbase-1b,2023-07-03 13:08:44+00:00,Unknown,7287,64,SafeTensorsInfo(parameters={'F32': 1137207296}...,text-generation,['bigcode/the-stack-dedup'],"[EvalResult(task_type='text-generation', datas...",['code_eval'],transformers,TransformersInfo(auto_model='AutoModelForCausa...,['GPTBigCodeForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'gp...",license: bigcode-openrail-m\nlibrary_name: tra...,1137207296,0.014044


In [17]:

save_filtered_df =False
if save_filtered_df:
    # Optional: Save the filtered DataFrame to a CSV file
    filtered_df.to_csv(save_dir + "filtered_models.csv", index=False)
    print(f"Filtered CSV file saved: {save_dir}filtered_models.csv")

In [18]:
# 
import random
i = random.randint(0,len(filtered_df))
print(i)
row = filtered_df.iloc[i]
row

24


model                       TheBloke/Phind-CodeLlama-34B-Python-v1-GPTQ
id                          TheBloke/Phind-CodeLlama-34B-Python-v1-GPTQ
created_at                                    2023-08-26 13:39:11+00:00
downloads_all_time                                              Unknown
downloads                                                            43
likes                                                                 7
safetensors           SafeTensorsInfo(parameters={'I32': 4156200960,...
task                                                    text-generation
datasets                                                            NaN
eval_results          [EvalResult(task_type='text-generation', datas...
metrics                                                             NaN
library_name                                               transformers
transformers_info     TransformersInfo(auto_model='AutoModelForCausa...
architecture                                       ['LlamaForCau

# stratified sampling

In [27]:
save_dir = 'model_selection/results/'
file_name = save_dir+'filtered_models.csv'
df = pd.read_csv(file_name)
print(f'total models: {len(df)}')

total models: 69


In [28]:
df.columns

Index(['model', 'id', 'created_at', 'downloads_all_time', 'downloads', 'likes',
       'safetensors', 'task', 'datasets', 'eval_results', 'metrics',
       'library_name', 'transformers_info', 'architecture', 'tags',
       'card_data', 'num_parameters', 'popularity'],
      dtype='object')

In [29]:
# add likes: popularity = sum of the normalized likes and downloads
stratification_criteria = ['popularity', 'num_parameters','created_at']

In [30]:
import pandas as pd

# Assuming you have loaded your DataFrame 'df'

# Convert 'created_at' to a numerical format for quartile calculation
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['created_at_numeric'] = df['created_at'].astype(int) // 10**9  # Convert to Unix timestamp, seconds

# Create quartiles for stratified sampling
df['popularity_quartile'] = pd.qcut(df['popularity'], q=2, labels=False)  # 2 quartiles
df['created_at_quartile'] = pd.qcut(df['created_at_numeric'], q=2, labels=False)  # 2 quartiles
df['num_parameters_quartile'] = pd.qcut(df['num_parameters'], q=3, labels=False)  # 3 quartiles

# Display the DataFrame with new quartile columns
print(df[['popularity_quartile', 'created_at_quartile', 'num_parameters_quartile']].head())


   popularity_quartile  created_at_quartile  num_parameters_quartile
0                    1                    1                        1
1                    1                    1                        1
2                    1                    0                        0
3                    1                    0                        0
4                    1                    0                        0


In [33]:
# Group by the quartile columns to create strata
strata = df.groupby(['popularity_quartile', 'created_at_quartile', 'num_parameters_quartile'])

# Select one model from each stratum
selected_models = strata.apply(lambda x: x.sample(1)).reset_index(drop=True)

# Display the selected models

save_selected_df =True 

if save_selected_df:
    # Optional: Save the filtered DataFrame to a CSV file
    selected_models.to_csv(save_dir + "selected_models_01.csv", index=False)
    print(f"Filtered CSV file saved: {save_dir}selected_models.csv")
    
selected_models

Filtered CSV file saved: model_selection/results/selected_models.csv


/tmp/ipykernel_169904/1651244523.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_models = strata.apply(lambda x: x.sample(1)).reset_index(drop=True)


,model,id,created_at,downloads_all_time,downloads,likes,safetensors,task,datasets,eval_results,...,transformers_info,architecture,tags,card_data,num_parameters,popularity,created_at_numeric,popularity_quartile,created_at_quartile,num_parameters_quartile
0,TheBloke/speechless-code-mistral-7B-v1.0-GPTQ,TheBloke/speechless-code-mistral-7B-v1.0-GPTQ,2023-10-13 06:08:01+00:00,Unknown,19,2,"SafeTensorsInfo(parameters={'I32': 880476160, ...",text-generation,"['jondurbin/airoboros-2.2', 'Open-Orca/OpenOrc...","[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['MistralForCausalLM'],"['transformers', 'safetensors', 'mistral', 'te...",language:\n- en\nlicense: llama2\nlibrary_name...,1198788608,6.558938e-03,1697177281,0,0,0
1,TheBloke/WizardCoder-Python-13B-V1.0-AWQ,TheBloke/WizardCoder-Python-13B-V1.0-AWQ,2023-09-19 00:53:10+00:00,Unknown,22,0,"SafeTensorsInfo(parameters={'I32': 1598361600,...",text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",license: llama2\nlibrary_name: transformers\nt...,2025589760,1.807119e-06,1695084790,0,0,1
2,vodkaslime/test-repo-stablecode,vodkaslime/test-repo-stablecode,2023-08-23 07:50:01+00:00,Unknown,33,0,SafeTensorsInfo(parameters={'BF16': 2769311040...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['GPTNeoXForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'gp...",language:\n- code\nlicense: other\ntags:\n- ca...,3306181952,2.710678e-06,1692777001,0,0,2
3,TechxGenus/starcoder2-3b-GPTQ,TechxGenus/starcoder2-3b-GPTQ,2024-03-22 13:35:25+00:00,Unknown,945,1,"SafeTensorsInfo(parameters={'I32': 363432960, ...",text-generation,['bigcode/the-stack-v2-train'],"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['Starcoder2ForCausalLM'],"['transformers', 'safetensors', 'starcoder2', ...",license: bigcode-openrail-m\nlibrary_name: tra...,688945152,3.356312e-03,1711114525,0,1,0
4,TechxGenus/starcoder2-15b-AWQ,TechxGenus/starcoder2-15b-AWQ,2024-04-12 14:03:22+00:00,Unknown,28,1,"SafeTensorsInfo(parameters={'I32': 1933885440,...",text-generation,['bigcode/the-stack-v2-train'],"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['Starcoder2ForCausalLM'],"['transformers', 'safetensors', 'starcoder2', ...",license: bigcode-openrail-m\nlibrary_name: tra...,2661535744,3.280988e-03,1712930602,0,1,1
5,nlpguy/granite-3.0-3b-a800m-base,nlpguy/granite-3.0-3b-a800m-base,2024-10-31 14:29:25+00:00,Unknown,5,0,SafeTensorsInfo(parameters={'F32': 3374286336}...,text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['GraniteMoeForCausalLM'],"['transformers', 'safetensors', 'granitemoe', ...",license: apache-2.0\nlibrary_name: transformer...,3374286336,4.107089e-07,1730384965,0,1,2
6,TheBloke/WizardCoder-Python-7B-V1.0-AWQ,TheBloke/WizardCoder-Python-7B-V1.0-AWQ,2023-09-19 00:52:16+00:00,Unknown,118,2,"SafeTensorsInfo(parameters={'I32': 815824896, ...",text-generation,NaN,"[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",license: llama2\nlibrary_name: transformers\nt...,1128837120,6.567070e-03,1695084736,1,0,0
7,smallcloudai/Refact-1_6B-fim,smallcloudai/Refact-1_6B-fim,2023-08-29 15:48:36+00:00,Unknown,28033,129,SafeTensorsInfo(parameters={'BF16': 1585842176...,text-generation,"['bigcode/the-stack-dedup', 'rombodawg/2XUNCEN...","[EvalResult(task_type='text-generation', datas...",...,TransformersInfo(auto_model='AutoModelForCausa...,['GPTRefactForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'gp...",language:\n- en\nlicense: bigscience-op

In [32]:
import pandas as pd

used_models = ["codeparrot/codeparrot-small",
               "EleutherAI/pythia-410m",
               "EleutherAI/pythia-1.4b",
               "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
               "microsoft/phi-2"
               ]

# Assuming 'df' is your DataFrame and you have a specific model ID
model_id = used_models[1]  # Replace with the actual model ID

for model_id in used_models[1:]: 
    # Filter the DataFrame to find the model with the given ID
    model = df[df['id'] == model_id]
    

    # Get the quartile values for the selected model
    popularity_quartile = model['popularity_quartile']
    created_at_quartile = model['created_at_quartile']
    num_parameters_quartile = model['num_parameters_quartile']

    # Display the quartile values
    print("model: ", model['id'])
    print("popularity Quartile:", popularity_quartile)
    print("Created At Quartile:", created_at_quartile)
    print("Num Parameters Quartile:", num_parameters_quartile)
    print("---------------------")


model:  Series([], Name: id, dtype: object)
popularity Quartile: Series([], Name: popularity_quartile, dtype: int64)
Created At Quartile: Series([], Name: created_at_quartile, dtype: int64)
Num Parameters Quartile: Series([], Name: num_parameters_quartile, dtype: int64)
---------------------
model:  Series([], Name: id, dtype: object)
popularity Quartile: Series([], Name: popularity_quartile, dtype: int64)
Created At Quartile: Series([], Name: created_at_quartile, dtype: int64)
Num Parameters Quartile: Series([], Name: num_parameters_quartile, dtype: int64)
---------------------
model:  Series([], Name: id, dtype: object)
popularity Quartile: Series([], Name: popularity_quartile, dtype: int64)
Created At Quartile: Series([], Name: created_at_quartile, dtype: int64)
Num Parameters Quartile: Series([], Name: num_parameters_quartile, dtype: int64)
---------------------
model:  Series([], Name: id, dtype: object)
popularity Quartile: Series([], Name: popularity_quartile, dtype: int64)
Crea